In [3]:
token = 'INTC'
token_indicators = f'{token}-INDICATORS'

In [4]:
import yfinance as yf

data = yf.download(token, interval='1d', period='max')
data = data.droplevel('Ticker', axis=1)
data.to_csv(f'../data/{token}.csv')
data.tail()

/var/folders/vp/2hwjkxh92qsdfnjr9y_xpmm00000gn/T/ipykernel_1338/3177959979.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(token, interval='1d', period='max')
[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Date,,,,,
2025-08-18,23.660000,24.900000,23.209999,24.000000,212515600
2025-08-19,25.309999,26.530001,24.940001,24.940001,297659200
2025-08-20,23.540001,24.930000,23.330000,24.900000,161149400
2025-08-21,23.500000,23.570000,22.780001,23.530001,97718300
2025-08-22,24.799999,25.230000,23.650000,23.650000,197891400


In [5]:
import pandas as pd
from ta.trend import EMAIndicator, MACD
from ta.momentum import RSIIndicator

# Carga tus datos en 'data' con columna 'Close'
data = pd.read_csv(f'../data/{token}.csv')

# EMAs
data['ema20'] = EMAIndicator(data['Close'], window=20).ema_indicator()
data['ema50'] = EMAIndicator(data['Close'], window=50).ema_indicator()

# RSIs
data['rsi10']  = RSIIndicator(data['Close'], window=10).rsi()
data['rsi30']  = RSIIndicator(data['Close'], window=30).rsi()

# MACD
macd_ind = MACD(data['Close'], window_fast=12, window_slow=26, window_sign=9)
data['macd']   = macd_ind.macd()
data['signal'] = macd_ind.macd_signal()
data['hist']   = macd_ind.macd_diff()

# Señal de subida de precio
data['Price-Up'] = (data['Close'].shift(-1) > data['Close']).astype(int)

# Limpieza y exportación
data.dropna(inplace=True)
data.to_csv(f'../data/{token_indicators}.csv', index=False)
data
print(data.tail())


             Date      Close       High        Low       Open     Volume  \
11448  2025-08-18  23.660000  24.900000  23.209999  24.000000  212515600   
11449  2025-08-19  25.309999  26.530001  24.940001  24.940001  297659200   
11450  2025-08-20  23.540001  24.930000  23.330000  24.900000  161149400   
11451  2025-08-21  23.500000  23.570000  22.780001  23.530001   97718300   
11452  2025-08-22  24.799999  25.230000  23.650000  23.650000  197891400   

           ema20      ema50      rsi10      rsi30      macd    signal  \
11448  21.868025  21.684996  65.022168  56.250773  0.348019 -0.121289   
11449  22.195832  21.827153  72.658591  60.333936  0.574202  0.017809   
11450  22.323848  21.894324  57.655400  54.671543  0.603670  0.134981   
11451  22.435863  21.957292  57.357997  54.551851  0.616687  0.231322   
11452  22.661019  22.068770  64.053620  57.667690  0.723562  0.329770   

           hist  Price-Up  
11448  0.469308         1  
11449  0.556393         0  
11450  0.468689     

# Stream_Crypto_Data 

In [7]:
import json 
import websocket
import pandas as pd
import numpy as np

coin = 'btcusdt@kline_1m'
socket = 'wss://stream.binance.com:9443/stream?streams=' + coin
print(socket)


ModuleNotFoundError: No module named 'websocket'

In [ ]:
def extract_ohlcv(source):
    event_time = pd.to_datetime(source['data']['E'], unit='ms')
    open = source['data']['k']['o']
    high = source['data']['k']['h']
    low = source['data']['k']['l']
    close = source['data']['k']['c']   
    volume = source['data']['k']['v']

    data = {'Open': open, 'High': high, 'Low': low, 'Close': close, 'Volume': volume}

    df = pd.DataFrame(data, index=[event_time])
    df.index.name = 'timestamp'
    df = df.astype(float)
    df = df.reset_index()
    print(df)
    return df

def on_message(ws, message):
    message = json.loads(message)
    extract_ohlcv(message)

In [ ]:
ws = websocket.WebSocketApp(socket, on_message=on_message)
ws.run_forever()

                timestamp      Open     High       Low     Close    Volume
0 2025-02-27 17:53:18.014  84573.53  84593.7  84494.26  84494.26  10.41565
                timestamp      Open     High       Low     Close    Volume
0 2025-02-27 17:53:20.015  84573.53  84593.7  84494.26  84494.27  10.43499
                timestamp      Open     High       Low     Close    Volume
0 2025-02-27 17:53:22.029  84573.53  84593.7  84494.26  84494.27  10.61498


True